# 어뷰징 판매자 탐지 모델 개발

쿠팡 판매자 데이터를 기반으로 어뷰징 판매자를 탐지하는 머신러닝 모델을 개발합니다.

이 노트북은 `03_feature_engineering.ipynb`에서 생성된 피처를 로드하여 모델을 학습합니다.

In [ ]:
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (
    RandomForestClassifier, 
    GradientBoostingClassifier,
    AdaBoostClassifier,
    ExtraTreesClassifier
)
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, roc_curve
)
import joblib
import warnings
warnings.filterwarnings('ignore')

# XGBoost (설치되어 있는 경우)
try:
    from xgboost import XGBClassifier
    HAS_XGBOOST = True
    print("XGBoost 사용 가능")
except ImportError:
    HAS_XGBOOST = False
    print("XGBoost 미설치 - 제외됨")

# LightGBM (설치되어 있는 경우)
try:
    from lightgbm import LGBMClassifier
    HAS_LIGHTGBM = True
    print("LightGBM 사용 가능")
except ImportError:
    HAS_LIGHTGBM = False
    print("LightGBM 미설치 - 제외됨")

import sys
sys.path.insert(0, os.path.abspath('..'))
from src.features.feature_generation import FEATURE_NAMES_KO, get_feature_name_ko

print("\n라이브러리 로드 완료")

## 1. 데이터 로드

`03_feature_engineering.ipynb`에서 생성된 피처 데이터를 로드합니다.

In [ ]:
# 피처 데이터 로드
FEATURE_PATH = '../data/processed/features.csv'

if not os.path.exists(FEATURE_PATH):
    print(f"오류: {FEATURE_PATH} 파일이 없습니다. 03_feature_engineering.ipynb를 먼저 실행해주세요.")
else:
    features_df = pd.read_csv(FEATURE_PATH)
    print(f"최종 피처 데이터 로드 완료: {features_df.shape}")
    
    # 피처 목록 (company_name, is_abusing_seller 제외한 모든 컬럼)
    exclude_cols = ['company_name', 'is_abusing_seller']
    feature_columns = [col for col in features_df.columns if col not in exclude_cols]
    
    print(f"총 피처 수: {len(feature_columns)}개")
    print(f"피처 목록: {feature_columns}")

## 2. 데이터 분할 및 전처리

In [ ]:
# 피처와 타겟 분리
X = features_df[feature_columns]
y = features_df['is_abusing_seller'].astype(int)

# X: 데이터, y: 정답 레이블 (총 401개)

# 1단계: 전체 -> 나머지(90%) / 최후검증(10%)
X_remain, X_final, y_remain, y_final = train_test_split(
    X, y, 
    test_size=0.1, 
    random_state=42, 
    stratify=y 
)

# 2단계: 나머지 -> 학습(60%) / 테스트(30%)
X_train, X_test, y_train, y_test = train_test_split(
    X_remain, y_remain, 
    test_size=1/3, 
    random_state=42,
    stratify=y_remain
)

print(f"훈련 세트: {X_train.shape[0]}개 (어뷰징: {y_train.sum()}개)")
print(f"테스트 세트: {X_test.shape[0]}개 (어뷰징: {y_test.sum()}개)")

In [ ]:
# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("스케일링 완료")

## 3. 모델 학습

In [ ]:
# 모델 평가 함수
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """모델 평가 및 결과 반환"""
    # 예측
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    y_test_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
    
    # 메트릭 계산
    results = {
        'model': model_name,
        'train_accuracy': accuracy_score(y_train, y_train_pred),
        'test_accuracy': accuracy_score(y_test, y_test_pred),
        'precision': precision_score(y_test, y_test_pred, zero_division=0),
        'recall': recall_score(y_test, y_test_pred, zero_division=0),
        'f1': f1_score(y_test, y_test_pred, zero_division=0),
        'roc_auc': roc_auc_score(y_test, y_test_proba) if y_test_proba is not None else None
    }
    
    return results, y_test_pred, y_test_proba

In [ ]:
# 모델 정의
models_config = {
    'Logistic Regression': {
        'model': LogisticRegression(random_state=42, max_iter=1000),
        'needs_scaling': True
    },
    'Random Forest': {
        'model': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
        'needs_scaling': False
    },
    'Gradient Boosting': {
        'model': GradientBoostingClassifier(n_estimators=100, random_state=42),
        'needs_scaling': False
    },
    'SVM': {
        'model': SVC(probability=True, random_state=42, kernel='rbf'),
        'needs_scaling': True
    },
    'KNN': {
        'model': KNeighborsClassifier(n_neighbors=5),
        'needs_scaling': True
    },
    'AdaBoost': {
        'model': AdaBoostClassifier(n_estimators=100, random_state=42),
        'needs_scaling': False
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(random_state=42, max_depth=10),
        'needs_scaling': False
    },
    'Extra Trees': {
        'model': ExtraTreesClassifier(n_estimators=100, random_state=42, n_jobs=-1),
        'needs_scaling': False
    },
    'Naive Bayes': {
        'model': GaussianNB(),
        'needs_scaling': True
    },
}

# XGBoost 추가 (설치된 경우)
if HAS_XGBOOST:
    models_config['XGBoost'] = {
        'model': XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='logloss'),
        'needs_scaling': False
    }

# LightGBM 추가 (설치된 경우)
if HAS_LIGHTGBM:
    models_config['LightGBM'] = {
        'model': LGBMClassifier(n_estimators=100, random_state=42, verbose=-1),
        'needs_scaling': False
    }

print(f"총 {len(models_config)}개 모델 준비 완료")

In [ ]:
# 모델 학습 및 평가
models = {}
results_list = []

print("="*60)
print("모델 학습 시작")
print("="*60)

for i, (name, config) in enumerate(models_config.items(), 1):
    print(f"\n{i}. {name} 학습 중...")
    
    model = config['model']
    needs_scaling = config['needs_scaling']
    
    # 스케일링 필요 여부에 따라 데이터 선택
    X_tr = X_train_scaled if needs_scaling else X_train
    X_te = X_test_scaled if needs_scaling else X_test
    
    # 학습
    model.fit(X_tr, y_train)
    models[name] = {'model': model, 'needs_scaling': needs_scaling}
    
    # 평가
    results, _, _ = evaluate_model(model, X_tr, X_te, y_train, y_test, name)
    results_list.append(results)
    
    print(f"   ✓ Test Accuracy: {results['test_accuracy']:.4f}, F1: {results['f1']:.4f}, ROC-AUC: {results['roc_auc']:.4f}")

print("\n" + "="*60)
print("모든 모델 학습 완료!")
print("="*60)

In [ ]:
# 결과 비교
results_df = pd.DataFrame(results_list)
print("\n=== 모델 성능 비교 ===")
print(results_df.to_string(index=False))

## 4. 모델 평가 시각화

In [ ]:
# 성능 비교 차트 (점수 표시 + 1등 강조)
metrics = ['test_accuracy', 'precision', 'recall', 'f1']
metric_names = ['정확도', '정밀도', '재현율', 'F1-Score']

# 각 메트릭별 1등 모델 찾기
best_per_metric = {m: results_df.loc[results_df[m].idxmax(), 'model'] for m in metrics}

# 색상 팔레트 (모델 수에 맞게 확장)
colors = px.colors.qualitative.Set3 + px.colors.qualitative.Pastel
model_colors = {model: colors[i % len(colors)] for i, model in enumerate(results_df['model'])}

fig = go.Figure()

for _, row in results_df.iterrows():
    model_name = row['model']
    scores = [row[m] for m in metrics]
    
    # 1등인 메트릭에만 테두리 강조
    marker_line_widths = []
    marker_line_colors = []
    for m in metrics:
        if best_per_metric[m] == model_name:
            marker_line_widths.append(3)
            marker_line_colors.append('gold')
        else:
            marker_line_widths.append(0)
            marker_line_colors.append('rgba(0,0,0,0)')
    
    fig.add_trace(go.Bar(
        name=model_name,
        x=metric_names,
        y=scores,
        text=[f'{s:.3f}' for s in scores],
        textposition='outside',
        textfont=dict(size=9),
        marker_color=model_colors[model_name],
        marker_line_width=marker_line_widths,
        marker_line_color=marker_line_colors,
    ))

fig.update_layout(
    title='모델별 성능 비교 - ⭐ 금테두리: 1등',
    barmode='group',
    yaxis_title='Score',
    yaxis_range=[0, 1.15],
    template='plotly_white',
    height=500,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5)
)
fig.show()

In [ ]:
# 최고 성능 모델 선택 (F1 기준)
best_model_name = results_df.loc[results_df['f1'].idxmax(), 'model']
best_model_info = models[best_model_name]
best_model = best_model_info['model']

print(f"최고 성능 모델: {best_model_name}")
print(f"  - F1-Score: {results_df.loc[results_df['f1'].idxmax(), 'f1']:.4f}")
print(f"  - ROC-AUC: {results_df.loc[results_df['f1'].idxmax(), 'roc_auc']:.4f}")

# 테스트 데이터 선택
X_test_final = X_test_scaled if best_model_info['needs_scaling'] else X_test

# 혼동 행렬
y_pred = best_model.predict(X_test_final)
cm = confusion_matrix(y_test, y_pred)

fig = go.Figure(data=go.Heatmap(
    z=cm,
    x=['정상 예측', '어뷰징 예측'],
    y=['정상 실제', '어뷰징 실제'],
    text=cm,
    texttemplate='%{text}',
    colorscale='Blues'
))

fig.update_layout(
    title=f'{best_model_name} - 혼동 행렬',
    template='plotly_white'
)
fig.show()

In [ ]:
# ROC Curve (모든 모델 비교)
fig = go.Figure()

# 색상 팔레트
colors = px.colors.qualitative.Set2 + px.colors.qualitative.Set1

for i, (name, model_info) in enumerate(models.items()):
    model = model_info['model']
    needs_scaling = model_info['needs_scaling']
    X_te = X_test_scaled if needs_scaling else X_test
    
    y_proba = model.predict_proba(X_te)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc = roc_auc_score(y_test, y_proba)
    
    fig.add_trace(go.Scatter(
        x=fpr, y=tpr,
        name=f'{name} (AUC={auc:.3f})',
        mode='lines',
        line=dict(color=colors[i % len(colors)])
    ))

fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    name='Random',
    mode='lines',
    line=dict(dash='dash', color='gray')
))

fig.update_layout(
    title='ROC Curve 비교 (모든 모델)',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    template='plotly_white',
    height=600,
    legend=dict(x=1.02, y=0.5, xanchor='left')
)
fig.show()

## 5. 피처 중요도 분석

In [ ]:
# Random Forest 피처 중요도
rf_model = models['Random Forest']['model']

feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'feature_ko': [get_feature_name_ko(f) for f in feature_columns],
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=True)

fig = go.Figure(data=go.Bar(
    x=feature_importance['importance'],
    y=feature_importance['feature_ko'],
    orientation='h',
    marker_color='#636EFA'
))

fig.update_layout(
    title='Random Forest - 피처 중요도',
    xaxis_title='중요도',
    yaxis_title='피처',
    height=600,
    template='plotly_white'
)
fig.show()

print("\n=== Top 10 중요 피처 ===")
top10 = feature_importance.tail(10)[['feature_ko', 'importance']].copy()
top10.columns = ['피처', '중요도']
print(top10.to_string(index=False))

## 6. 모델 저장

In [ ]:
# models 디렉토리 생성
os.makedirs('../models', exist_ok=True)

# 최고 성능 모델 저장
model_filename = f'abusing_detector_{best_model_name.lower().replace(" ", "_")}.pkl'
joblib.dump(best_model, f'../models/{model_filename}')
joblib.dump(scaler, '../models/scaler.pkl')

# 피처 목록 저장
with open('../models/feature_columns.txt', 'w') as f:
    f.write('\n'.join(feature_columns))

# 모델 성능 비교 결과 저장
results_df.to_csv('../models/model_comparison.csv', index=False)

print(f"모델 저장 완료: models/{model_filename}")
print("스케일러 저장 완료: models/scaler.pkl")
print("피처 목록 저장 완료: models/feature_columns.txt")
print("성능 비교 저장 완료: models/model_comparison.csv")